##### COORESPONDANCE DES FICHIERS COLLECTES

In [15]:
import pandas as pd
from datetime import datetime, timedelta
import os, sys
import numpy as np
import re
from sqlalchemy import create_engine
import logging

In [152]:
panier_df = pd.read_excel("Echantillon_Data_Scrapping_09032024_valide_code.xlsx")
panier_df.head()

,Date_de_collecte,Code_site,Libelle_du_produit,Code produit,Quantite,Prix_du_produit,Caracteristique,Unite,Unite_monetaire,Code,Observation
0,2024-03-09 04:02:44,adjovan,HARICOT BLANC VRAC [300G],1010703020301,300,500,Pas de Caracteristique,G,CFA,111111,Ce produit figure dans le panier de bien
1,2024-03-09 04:05:09,adjovan,CARTON DE COTE DE PORC (STERNUM) [10KG],1010203010201,10,13.000 12.000,Pas de Caracteristique,KG,CFA,111111,ce produit figure dans le panier de bien
2,2024-03-09 04:05:20,adjovan,COTES DE PORC SANS PEAU [500G],1010203010201,500,1.600,Le porc contient plusieurs nutriments essentie...,G,CFA,111111,ce produit figure dans le panier de bien
3,2024-03-09 04:05:23,adjovan,COTES/COTELETTES DE BOEUF FUMÉES [1/2KG],1010201010101,1,1.100,Pas de Caracteristique,KG,CFA,111111,ce produit figure dans le panier
4,2024-03-09 04:05:37,adjovan,ÉCHINE DE PORC [500G],1010203010201,500,1.500 1.000,"Tirée de la partie supérieure du cou, l’échine...",G,CFA,111111,ce produit figure dans le panier de bien


##### COORESPONDANCE AVEC LE PANIER DE BIEN

In [155]:
colonnes_a_comparer = ["Libelle_du_produit"] 

date_debut = datetime.strptime("01-03-2024", "%d-%m-%Y")  
date_fin = datetime.strptime("30-03-2024", "%d-%m-%Y")  
# Traiter les intervalles de prix pour obtenir le plus petit prix
def extract_min_price(value):
    if isinstance(value, str):
        if "-" in value:
            parts = value.split("-")
            if len(parts) >= 2:
                return int(parts[0].strip())
        elif " " in value:
            return int(value.split()[0])
    return value

while date_debut <= date_fin:
    date_str = date_debut.strftime("%d%m%Y")
    fichier_journalier = os.path.join("Data_Collecte","mars", f"Data_Scrapping_{date_str}.xlsx")
    try:
        df_journalier = pd.read_excel(fichier_journalier)
        
        lignes_correspondantes = pd.merge(df_journalier, panier_df[['Libelle_du_produit', 'Code produit', 'Code']], how="inner", on=colonnes_a_comparer)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].fillna("")
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: x.replace(".", "") if isinstance(x, str) else x)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: x.replace(",", "") if isinstance(x, str) else x)
        #lignes_correspondantes["Prix_du_produit"] = pd.to_numeric(lignes_correspondantes["Prix_du_produit"].str.replace("[^\d-]", "", regex=True), errors='coerce')
        #lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: float(x.split("-")[0]) if isinstance(x, str) and "-" in x else x)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(extract_min_price)
        
        fichier_sortie = os.path.join("Data_Valide_Prix","mars",f"Data_Scrapping_{date_str}.xlsx")
        lignes_correspondantes.to_excel(fichier_sortie, index=False)
        
    except FileNotFoundError:
        print(f"Le fichier {fichier_journalier} n'existe pas.")
 
    date_debut += timedelta(days=1)

Le fichier Data_Collecte\mars\Data_Scrapping_10032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_15032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_16032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_17032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_19032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_24032024.xlsx n'existe pas.


##### FUSION ET AGGREGATION DES DONNEES DU MOIS PRECEDENT(Mars)

In [4]:
#AJOUT DES TYPES DE VARIETE AUX COLLECTES JOURNALIERES
panier_df2 = pd.read_excel("Panier_renomme.xlsx")
def clean_and_convert(value):
    if pd.isna(value):
        return np.nan
    elif isinstance(value, (int, float)):
        return int(value)  
    elif isinstance(value, str):
        try:
            numeric_part = re.search(r'\d+', value).group()  
            return int(numeric_part) 
        except (TypeError, AttributeError, ValueError):
            return np.nan  
'''  
def mean_ignore_nan(values):
    if values.isnull().all():
        return np.nan
    return np.nanmean(values)
'''

date_debut = datetime.strptime("01-03-2024", "%d-%m-%Y")  
date_fin = datetime.strptime("30-03-2024", "%d-%m-%Y")

while date_debut <= date_fin:
    date_str = date_debut.strftime("%d%m%Y")
    fichier_journalier = os.path.join("Data_Valide_Prix","mars", f"Data_Scrapping_{date_str}.xlsx")
    try:
        df_journalier = pd.read_excel(fichier_journalier)
        
        df_journalier['Prix_du_produit'] = df_journalier['Prix_du_produit'].apply(clean_and_convert)
        df_journalier['Unite_monetaire'] ='CFA'
        #df_journalier['Date_de_collecte'] = pd.to_datetime(df_journalier['Date_de_collecte'])
        premiere_date = df_journalier.loc[0, 'Date_de_collecte'].split()[0]
        df_journalier['Date_de_collecte'] = premiere_date
        #donnees_groupees = df_journalier.groupby(['Date_de_collecte','Unite_monetaire','Libelle_du_produit']).agg({'Prix_du_produit': 'mean'}).reset_index()
        donnees_groupees2 = pd.merge(df_journalier, panier_df2[['Code produit','Type variété (HE, O1,O2,O3)']], how="inner", on="Code produit")
        #donnees_groupees = df_journalier.groupby(['Date_de_collecte', 'Unite_monetaire', 'Code produit']).agg({'Prix_du_produit': mean_ignore_nan}).reset_index()
        #print(f"Data_Scrapping_{date_str}.xlsx")
        fichier_sortie = os.path.join("Data_correction1","mars",f"Data_Scrapping_{date_str}.xlsx")
        donnees_groupees2.to_excel(fichier_sortie, index=False)
            
    except FileNotFoundError:
        print(f"Le fichier {fichier_journalier} n'existe pas.")
 
    date_debut += timedelta(days=1)

Le fichier Data_Valide_Prix\mars\Data_Scrapping_10032024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\mars\Data_Scrapping_15032024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\mars\Data_Scrapping_16032024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\mars\Data_Scrapping_17032024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\mars\Data_Scrapping_19032024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\mars\Data_Scrapping_24032024.xlsx n'existe pas.


In [5]:
#FUSION DE TOUS LES FICHIERS DU MOIS PRECEDENT
dossier = os.path.join("Data_correction1","mars")
dfs = []

for fichier in os.listdir(dossier):
        
        chemin_complet = os.path.join(dossier, fichier)
        df = pd.read_excel(chemin_complet)
        dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)

path_out = os.path.join("Data_correction1","mars","F_Data_Scrapping_Mars.xlsx")
df_final.to_excel(path_out, index=False)


In [8]:
#AGGREGATION DES DONNEES DU FICHIER DU MOIS PAR PRIX ET AJOUT DE LA COLONNE "FOURCHETTE".

# Calculer les fourchettes pour chaque produit
def calculate_price_ranges(row):
    price = row['Prix_du_produit']
    
    if row['Type variété (HE, O1,O2,O3)'] == 'Heterogene':
        lower_bound = price * 0.85
        upper_bound = price * 1.15
    elif row['Type variété (HE, O1,O2,O3)'] == 'Homogene':
        lower_bound = price * 0.70
        upper_bound = price * 1.30
    else:
        raise ValueError(f"Variété non reconnue pour le produit {row['Libelle_du_produit']}")
    
    return f"{lower_bound:.2f}-{upper_bound:.2f}"

path1 = os.path.join("Data_correction1","mars","F_Data_Scrapping_Mars.xlsx")
df_FDSM = pd.read_excel(path1)
df_FDSM['Unite_monetaire'] ='CFA'
premiere_date = df_FDSM.loc[0, 'Date_de_collecte'].split()[0]
df_FDSM['Date_de_collecte'] = premiere_date
df_FDSM_A = df_FDSM.groupby(['Date_de_collecte','Libelle_du_produit','Unite_monetaire','Type variété (HE, O1,O2,O3)']).agg({'Prix_du_produit': 'mean'}).reset_index()
df_FDSM_A['fourchette'] =df_FDSM_A.apply(calculate_price_ranges, axis=1)
path_out = os.path.join("Data_correction1","mars","A_Data_Scrapping_Avril.xlsx")
df_FDSM_A.to_excel(path_out, index=False)

##### CORRECTION 

##### Control sur les prix selon la fourchette attribuée à la variété

In [11]:
previous_month_file = os.path.join("Data_correction1","mars","A_Data_Scrapping_Avril.xlsx")

aggregated_df = pd.read_excel(previous_month_file)

# Créer un dictionnaire pour un accès rapide aux fourchettes
price_ranges = {}

for _, row in aggregated_df.iterrows():
    lower_bound, upper_bound = map(float, row['fourchette'].split('-')) 
    price_ranges[row['Libelle_du_produit']] = (lower_bound, upper_bound)

# Fonction pour vérifier si le prix est dans la fourchette
def is_within_range(product, price):
    if product not in price_ranges:
        return None
    lower_bound, upper_bound = price_ranges[product]
    return lower_bound <= price <= upper_bound

# Répertoire contenant les fichiers journaliers du mois suivant
daily_files_directory= os.path.join("Data_correction1","avril")

# Parcourir chaque fichier journalier
for daily_file in os.listdir(daily_files_directory):
    if daily_file.endswith('.xlsx') : 
        daily_file_path = os.path.join(daily_files_directory, daily_file)
        daily_df = pd.read_excel(daily_file_path)

        # Créer une liste des lignes à conserver
        rows_to_keep = []

        for _, row in daily_df.iterrows():
            product = row['Libelle_du_produit']
            if product in price_ranges:
                price = row['Prix_du_produit']
                if pd.notna(price) and not is_within_range(product, price):
                    row['Prix_du_produit'] = None
                rows_to_keep.append(row)
            else:
                # Produit non présent dans le fichier du mois précédent
                continue

        # Convertir la liste des lignes à conserver en DataFrame
        cleaned_df = pd.DataFrame(rows_to_keep, columns=daily_df.columns)

        # Sauvegarder le fichier nettoyé
        cleaned_file_path = os.path.join("Data_Correction2","avril",daily_file)
        cleaned_df.to_excel(cleaned_file_path, index=False)


##### TRIATEMENT D'IMPUTATION DES PRIX MANQUANTS

In [16]:
# Configurer le journal
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_prev_month = os.path.join("Data_correction1","mars","A_Data_Scrapping_Avril.xlsx")
df_prev_month = pd.read_excel(file_prev_month)

logging.info("Fichier agrégé du mois précédent chargé.")

# Fonction pour l'imputation des prix manquants
def impute_missing_prices(df_daily, df_prev_month):
    for index, row in df_daily.iterrows():
        if pd.isna(row["Prix_du_produit"]):
            product = row["Libelle_du_produit"]
            prev_price_row = df_prev_month[df_prev_month["Libelle_du_produit"] == product]
            if not prev_price_row.empty:
                prev_price = prev_price_row["Prix_du_produit"].values[0]
                variety = prev_price_row["Type variété (HE, O1,O2,O3)"].values[0]
                
                # Filtrer les produits de la même variété dans le fichier journalier
                same_variety_products = df_daily[df_daily["Type variété (HE, O1,O2,O3)"] == variety]
                # Exclure les produits avec des prix manquants
                same_variety_products = same_variety_products.dropna(subset=["Prix_du_produit"])
                
                if not same_variety_products.empty:
                    # Calculer les variations individuelles de chaque produit de la même variété
                    variations = []
                    for _, same_variety_row in same_variety_products.iterrows():
                        same_variety_product = same_variety_row["Libelle_du_produit"]
                        prev_same_variety_row = df_prev_month[df_prev_month["Libelle_du_produit"] == same_variety_product]
                        if not prev_same_variety_row.empty:
                            prev_same_variety_price = prev_same_variety_row["Prix_du_produit"].values[0]
                            current_price = same_variety_row["Prix_du_produit"]
                            variation = abs(current_price - prev_same_variety_price)
                            variations.append(variation)
                    
                    # Calculer la variation moyenne
                    if variations:
                        avg_variation = sum(variations) / len(variations)
                        new_price = prev_price * avg_variation
                        df_daily.at[index, "Prix_du_produit"] = new_price
                        logging.info(f"Imputation faite pour {product} avec le nouveau prix {new_price}.")
                else:

                    logging.info(f"Aucun produit similaire trouvé pour la variété {variety}.")

# Parcourir les fichiers journaliers dans le répertoire
daily_files_directory = os.path.join("Data_Correction2","avril")
output_directory = "imputation_directory"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

logging.info("Début du traitement des fichiers journaliers.")

for file_name in os.listdir(daily_files_directory):
    if file_name.endswith(".xlsx"):
        daily_file_path = os.path.join(daily_files_directory, file_name)
        df_daily = pd.read_excel(daily_file_path)

        logging.info(f"Traitement du fichier {file_name}.")

        impute_missing_prices(df_daily, df_prev_month)
        
        corrected_file_path = os.path.join(output_directory, f"{file_name}_clean")
        df_daily.to_excel(corrected_file_path, index=False)


logging.info("Traitement terminé.")


2024-06-03 10:53:28,364 - INFO - Fichier agrégé du mois précédent chargé.
2024-06-03 10:53:28,368 - INFO - Début du traitement des fichiers journaliers.
2024-06-03 10:53:29,078 - INFO - Traitement du fichier Data_Scrapping_02042024.xlsx.
2024-06-03 10:53:29,535 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 94885.99736380122.
2024-06-03 10:53:29,918 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 431214.6656434737.
2024-06-03 10:53:30,318 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 2024813.8530419176.
2024-06-03 10:53:30,851 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 9571184.892919725.
2024-06-03 10:53:31,322 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 45285414.37738995.
2024-06-03 10:53:31,746 - INFO - Imputation faite pour Beurre doux avec le nouveau prix 148413458.68752244.
2024-06-03 10:53:32,163 - INFO - Imputation fait

KeyboardInterrupt: 

###